# Künstlicher Assistent für die Suche nach den Trainingsdaten

## Google Colab Arbeitplatz vorbereiten

In [ ]:
!git clone https://github.com/edadunashvili/Monographie.git

In [ ]:
cd /content/Monographie/ErthaosMinus

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Aggregation des Korpus von unbekannten Daten 

[Zelle # 1]

1-5: Der vorhandene Daten-Korpus löschen (falls vorhanden). Die Funktion wird im Folgenden zwecks Beseitigung von weiteren Aggregationen verwendet.

6-61: Aggregation des Rohdatenkorpus.

62-70: Aggregation der "episode_string_such.csv" Datei.

71-76: 'episode_string_such.csv' laden, von den überflüssigen Zeichen bereinigen, fehlende Feldwerte auffüllen und in 'episode_binar_such.csv' umwandeln.

77-78: Unnotige Aggregationen werden gelöscht.

In [14]:
def entfernen(var):
    import os
    if os.path.exists(var):
        os.remove(var)
entfernen('episode_binar_roh.csv')

In [15]:
def clean(line):
    line = line.replace("\n"," ").strip()
    line = line.replace("ä","ae").replace("ü","ue").replace("ö","oe")
    line = line.replace("ß","ss").replace(",","").replace("«","")
    line = line.replace("»","").replace(".","").replace(":","")
    line = line.replace(";","").replace('"',"")
    line = line.replace("?","").replace("!","").replace("á","a")
    line = line.replace(",","").replace("\t"," ").replace("'","")
    line = line.replace("‹","").replace("›","").replace("-"," ")
    line = line.replace("'('","").replace("')'","").replace('>','')
    line = line.replace("    "," ").replace("   "," ").replace("  "," ")
    line = line.replace('–','').replace('—','').replace('<','')
    line = line.replace("Â", "A").replace("ø", "oe").replace('“','')
    line = line.replace('„','').replace('(','').replace(')','')
    line = line.replace("‚", "").replace(']','').replace('[','')
    if line == "": 
        return
    line=line.split("|")
    line[0]=line[0].split("|")[0]
    for i, _ in enumerate(line):
        if (i !=0) and (i!=2):
            line[i]=line[i].lower()
    flex=[]
    try:
        flex=line[1].split("")
    except:
        pass
    value=str(line)
    line=str(line)   
    flex.append(line)
    ret=[]
    for i in flex:
        ret.append((i,value[0]))
    return ret
def write_back(words, temp):
    with open(temp, "a", encoding='utf-8') as output:      
        for word in words:
            as_lex = word[0]
            full_word = '"' + as_lex + '"'
            for sub_word in word[1:]:
                full_word += " , "  '"' + sub_word + '"'
            full_word +="\n"
            output.write(full_word)
def aggreg(temp, ordner):
    import glob
    with open(temp, "w", encoding='utf-8') as output:
        output.write ("quelle,episode,index_string,index_binar\n")
    sorted_files=sorted(glob.glob(ordner))
    pairs = []
    for file in sorted_files:
        with open(file, 'r', encoding='utf-8', errors='ignore') as episode:
            for line in episode.readlines():
                clean_words = clean(line)
                pairs = pairs + clean_words
    write_back(pairs, temp)
temp = "episode_temp_roh.csv"
ordner = "Datendepot/*.txt"
aggreg(temp, ordner)
fin = open('episode_temp_roh.csv','r', encoding ='utf-8')
fout = open('episode_string_roh.csv', "wt", encoding ='utf-8')
for kfz in fin:
    fout.write(kfz.replace(', "["',"").replace('"[', "").replace(']"',"")
               .replace("', '", "','").replace(" '", "'"))
fin.close()
fout.close()

fin = open('episode_string_roh.csv','r', encoding='utf-8') 
fout = open('episode_binar_roh.csv', "wt", encoding='utf-8')
for efz in fin:
    fout.write(efz.replace("'","").replace(',,0,0',',test,0,0'))             
fin.close()
fout.close()
entfernen('episode_temp_roh.csv')
entfernen('episode_string_roh.csv')

[Zelle # 2]

1-11: Rohdatenkopus aus dem Datenfeld 'epsode' bestimmen und alle andere Felder ignorieren.

12-17: Datenfeld 'episode' von Stoppwörter bereinigen, zur Auffüllung der fehlende Werte eingesetzte synthetischer Stoff (hier das Wort 'test') entfernen.  

In [16]:
import csv
import pandas as pd
df = pd.read_csv('episode_binar_roh.csv', encoding='utf-8')
t_corpus = []
file = 'episode_binar_roh.csv'
with open(file, "r", encoding='utf-8') as maerchen:
    maerchen = df.episode
    reader = csv.reader(maerchen, delimiter = ',')
    for row in reader:
        lst = str(row)
        t_corpus.append(lst)
custom_stop_word_list=['test']
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
NLTK_stop_words_list=stopwords.words('german')
de = custom_stop_word_list + NLTK_stop_words_list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elguj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Suche nach den Trainigsdaten mit Hife COSINE-SIMILARITY

[Zelle # 3]

1-14: Vektorisieren, Transformieren und die Metrik für die Vorhersagefunktion definieren

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer
vectorizers = vectorizer(analyzer='word', ngram_range=(1,1), 
                         min_df = 2, stop_words = de)
vokabular =  vectorizers.fit_transform(t_corpus)
metrik = cosine_similarity
def find_similar(vokabular, index, top_n =-1):   
    metriks = metrik(vokabular[index:index+1], vokabular).flatten()
    related_docs_indices = [i for i in metriks.argsort()[::-1] 
                            if i != index]
    return [(index, metriks[index]) for index 
            in related_docs_indices][0:top_n]

[Zelle # 4]

1-3: Gesuchte Episode (Zielepisode) eingeben und einblenden lassen.

Anmerkung: Die gesuchte Episode wird über die 'n_te'-Variable eingegeben (manuell eingepflegte Zielepisoden sollen deshalb immer in der ersten Reihen der Korpusdatensätzen stehen und alphanummerisch angeordnet sein. Wir bevorzugen, dass diese Daten mit dem Prefix '!_' + Kleinbuchstabe benannt wird.)


In [18]:
n_te = 0
synt = t_corpus[n_te]
print(df.quelle[n_te],'-', synt)

_a - [' als nun der jaeger mit den drei thieren weiter zog kam er in eine stadt die mit schwarzem flor umzogen war er vernahm dass ein drache in der gegend hause der alljaehrlich eine jungfrau verlange in diesem jahre sei die reihe an des koenigs tochter und darum trauere alles der koenig habe sie demjenigen zur frau versprochen der den drachen toedte']


[Zelle # 5]

1-3: Suchen starten und die Ergebnisse einblenden lassen

In [19]:
for index, score in (find_similar(vokabular, n_te)):
    if (df.index_string[index]==0 and score>0.19):
        print(index,'-', score,'-', df.quelle[index],'-', 
           t_corpus[index], '-',  df.index_binar[index],'-', df.index_binar[index],'\n' )

4541 - 0.28798370561836417 -  60_Die_zwei_Bruedertxt - ['der jaeger aber war traurig zog in der welt herum und liess seine tiere vor den leuten tanzen es trug sich zu dass er gerade nach verlauf eines jahres wieder in dieselbe stadt kam wo er die koenigstochter vom drachen erloest hatte und die stadt war diesmal ganz mit rotem scharlach ausgehaengt da sprach er zum wirt was will das sagen vorm jahr war die stadt mit schwarzem flor ueberzogen was soll heute der rote scharlach der wirt antwortete vorm jahr sollte unsers koenigs tochter dem drachen ausgeliefert werden aber der marschall hat mit ihm gekaempft und ihn getoetet und da soll morgen ihre vermaehlung gefeiert werden darum war die stadt damals mit schwarzem flor zur trauer und ist heute mit rotem scharlach zur freude ausgehaengt'] - 0 - 0 

3016 - 0.26564266112602974 -  4_Der_Jaeger_ueber_alle_Jaegertxt - ['damit war die muehle und die heerde von den riesen erloest und der jaeger ueber alle jaeger setzte seine reise fort er kam i

## Ende 